In [ ]:
pip install xgboost

In [1]:
import numpy as np
import pandas as pd
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import common
import matplotlib.pyplot as plt

In [34]:
# nba = common.get_cleaned_baseline_data('./data/cleaned_data/base_cleaned.csv')
nba = pd.read_csv('./data/cleaned_data/external_cleaned.csv')
nba = nba.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'college','archetype','all_star','all_nba', 'position','team'])
nba = nba.dropna()
nba = nba.loc[:, (nba != 0).any(axis=0)]
nba = common.add_log_y_values(nba)
nba

,playerName,seasonStartYear,salary,inflationAdjSalary,startYear,height,weight,MP,PTS,Age,...,conference_champ,mvp,mvp_rank,mvp_total,player_week_enc,player_week_total,dpoy,dpoy_rank,dpoy_total,inflationAdjSalary_log
0,Michael Jordan,1996,30140000,52258566,1985.0,78.0,198.0,3090.0,2491.0,32.0,...,1,1,1,4,2,20,0,6,0,17.771714
1,Horace Grant,1996,14857000,25759971,1988.0,82.0,215.0,2286.0,847.0,30.0,...,0,0,100000,0,0,0,0,8,0,17.064332
2,Reggie Miller,1996,11250000,19505934,1988.0,79.0,185.0,2621.0,1606.0,30.0,...,0,0,100000,0,1,2,0,100000,0,16.786229
3,Shaquille O'Neal,1996,10714000,18576585,1993.0,85.0,325.0,1946.0,1434.0,23.0,...,0,0,9,0,2,6,0,100000,0,16.737412
4,Gary Payton,1996,10212000,17706187,1991.0,76.0,180.0,3162.0,1563.0,27.0,...,1,0,6,0,0,1,1,1,1,16.689425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10461,Adreian Payne,2017,77250,85683,2015.0,82.0,237.0,135.0,63.0,25.0,...,0,0,100000,0,0,0,0,100000,0,11.358410
10465,James Young,2017,77250,85683,2015.0,78.0,215.0,220.0,68.0,21.0,...,0,0,100000,0,0,0,0,100000,0,11.358410
10471,Darrun Hilliard,2017,77250,85683,2016.0,78.0,205.0,381.0,127.0,23.0,...,0,0,100000,0,0,0,0,100000,0,11.358410
10481,Jonathan Gibson,2017,74158,82253,2017.0,74.0,185.0,231.0,106.0,29.0,...,0,0,100000,0,0,0,0,100000,0,11.317555


In [23]:
X_train, X_test, y_train, y_test = common.split_data(nba)

In [28]:
feature_names = nba.columns.values.tolist()
feature_names.remove('playerName')
feature_names.remove('inflationAdjSalary_log')
len(feature_names)

34


34

In [32]:
# train_dmatrix = xg.DMatrix(data = X_train, label = y_train, feature_names=feature_names)
# test_dmatrix = xg.DMatrix(data = X_test, label = y_test, feature_names=feature_names)
  
# # Parameter dictionary specifying base learner
# param = {"objective":"reg:linear"}
  
# xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 10)
# pred = xgb_r.predict(test_dmatrix)

xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123)
  
# Fitting the model
xgb_r.fit(X_train, y_train)
  
# Predict the model
pred = xgb_r.predict(X_test)
  
# RMSE Computation
rmse = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE : % f" %(rmse))

MSE = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"MSE: {MSE}, R2: {r2}")

[12:26:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
RMSE :  0.429284
MSE: 0.18428505009325258, R2: 0.8833291557934211


In [31]:
feature_important = xgb_r.get_score(importance_type='weight')
feature_important
# keys = list(feature_important.keys())
# values = list(feature_important.values())

# data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
# data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

{'salary': 4.0, 'inflationAdjSalary': 68.0, 'Age': 1.0}